In [175]:
import sys
sys.path.insert(1, 'G:/My Drive/Semester_4/IML/IML.HUJI')
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats

import IMLearn.learners.regressors.linear_regression
from IMLearn.learners.regressors import PolynomialFitting
from IMLearn.utils import split_train_test

In [176]:
df = pd.read_csv("../datasets/City_Temperature.csv", parse_dates=['Date']).dropna().drop_duplicates()
df

,Country,City,Date,Year,Month,Day,Temp
0,South Africa,Capetown,1995-01-01,1995,1,1,19.333333
1,South Africa,Capetown,1995-01-02,1995,1,2,19.888889
2,South Africa,Capetown,1995-01-03,1995,1,3,19.388889
3,South Africa,Capetown,1995-01-04,1995,1,4,20.833333
4,South Africa,Capetown,1995-01-05,1995,1,5,21.444444
...,...,...,...,...,...,...,...
32434,Jordan,Amman,2020-05-09,2020,5,9,17.555556
32435,Jordan,Amman,2020-05-10,2020,5,10,17.055556
32436,Jordan,Amman,2020-05-11,2020,5,11,20.666667
32437,Jordan,Amman,2020-05-12,2020,5,12,24.444444


In [177]:
df.dtypes

Country            object
City               object
Date       datetime64[ns]
Year                int64
Month               int64
Day                 int64
Temp              float64
dtype: object

In [178]:
# Load data:
df['DayOfYear'] = df['Date'].dt.dayofyear
# df.drop(df.loc[(df.Temp < -60) & (df.Country.astype(str) == "Israel")].index, inplace=True)
df = df[(np.abs(stats.zscore(df['Temp'])) < 5)]

In [179]:
# Q2
il_df = df.loc[df.Country.astype(str) == "Israel"].reset_index(drop=True)
il_df.Year = il_df.Year.astype(str)
il_df

,Country,City,Date,Year,Month,Day,Temp,DayOfYear
0,Israel,Tel Aviv,1995-01-01,1995,1,1,14.055556,1
1,Israel,Tel Aviv,1995-01-02,1995,1,2,13.388889,2
2,Israel,Tel Aviv,1995-01-03,1995,1,3,13.277778,3
3,Israel,Tel Aviv,1995-01-04,1995,1,4,13.833333,4
4,Israel,Tel Aviv,1995-01-05,1995,1,5,13.666667,5
...,...,...,...,...,...,...,...,...
4191,Israel,Tel Aviv,2007-09-11,2007,9,11,26.388889,254
4192,Israel,Tel Aviv,2007-09-12,2007,9,12,26.500000,255
4193,Israel,Tel Aviv,2007-09-13,2007,9,13,26.500000,256
4194,Israel,Tel Aviv,2007-09-14,2007,9,14,26.444444,257


In [180]:
px.scatter(il_df, x='DayOfYear', y='Temp', color='Year')

In [181]:
il_df.groupby('Month').agg({'Temp': ['std', 'mean', 'max', 'min']})#.reset_index().rename(columns={'Temp': 'Temp_STD'})

Temp                                 
            std       mean        max        min
Month                                           
1      1.857489  14.188667  20.888889   7.277778
2      2.051790  14.592180  24.000000   8.055556
3      2.571599  16.443837  31.055556  10.388889
4      2.592382  18.929997  31.388889  11.333333
5      1.788606  21.869905  29.666667  17.555556
6      1.263369  24.960507  29.944444  21.555556
7      0.855084  27.184755  29.944444  24.555556
8      0.804522  27.988680  30.833333  25.555556
9      1.125120  26.859133  29.777778  22.111111
10     2.004767  24.029401  30.388889  16.000000
11     2.110169  20.225761  27.611111  14.555556
12     2.035612  16.085292  22.055556   9.055556

In [182]:
px.bar(il_df.groupby('Month').agg({'Temp': 'std'}).reset_index().rename(columns={'Temp': 'Temp_STD'}),
       x='Month', y='Temp_STD', title="Standard Deviation of Daily Temperatures Per Month")

In [183]:
ndf = df.groupby(['Country', 'Month']).agg({'Temp': ['mean', 'std']}).reset_index()
ndf.columns = ndf.columns.droplevel(0)
ndf.columns = ['Country', 'Month', 'Temp_mean', 'Temp_std']
ndf
# ndf['Temp_mean'] = ndf['Temp']['mean']
# ndf['Temp_std'] = ndf['Temp']['std']
# ndf.drop('Temp', axis=1)

,Country,Month,Temp_mean,Temp_std
0,Israel,1,14.188667,1.857489
1,Israel,2,14.592180,2.051790
2,Israel,3,16.443837,2.571599
3,Israel,4,18.929997,2.592382
4,Israel,5,21.869905,1.788606
5,Israel,6,24.960507,1.263369
6,Israel,7,27.184755,0.855084
7,Israel,8,27.988680,0.804522
8,Israel,9,26.859133,1.125120
9,Israel,10,24.029401,2.004767


In [184]:
px.line(ndf, x='Month', y='Temp_mean', color='Country', error_y='Temp_std').update_xaxes(tick0=1, dtick=1)

In [185]:
train_x, train_y, test_x, test_y = split_train_test(il_df['DayOfYear'], il_df['Temp'])


In [186]:
mse_per_k = []
for k in range(1, 11):
    poly = PolynomialFitting(k)
    poly.fit(train_x.to_numpy(), train_y.to_numpy())
    mse = np.round(poly.loss(test_x.to_numpy(), test_y.to_numpy()), 3)
    mse_per_k.append((k, mse))
    print(f"k={k} \t loss = {mse_per_k[-1][1]}")



k=1 	 loss = 21.662
k=2 	 loss = 6.449
k=3 	 loss = 3.604
k=4 	 loss = 3.196
k=5 	 loss = 3.056
k=6 	 loss = 6.104
k=7 	 loss = 13.874
k=8 	 loss = 30.453
k=9 	 loss = 30.451
k=10 	 loss = 30.439


In [187]:
mse_per_k_df = pd.DataFrame(mse_per_k, columns=['k', 'MSE'])
px.bar(mse_per_k_df, x=mse_per_k_df['k'].astype(str), y='MSE', title="Test Error For Each Value of k", text='MSE',
       labels=dict(x='k')).show()

In [199]:
best_k = min(mse_per_k, key=lambda tup: tup[1])[0]
best_k

5

In [200]:
# fig = make_subplots(1, 4)

pol = PolynomialFitting(best_k)
pol.fit(il_df['DayOfYear'].to_numpy(), il_df['Temp'].to_numpy())
results = []
for country in df.Country.unique():
    country_df = df[['DayOfYear', 'Temp']].loc[df.Country.astype(str) == country].reset_index(drop=True)
    results.append((country, pol.loss(country_df.DayOfYear.to_numpy(), country_df.Temp)))
results_df = pd.DataFrame(results, columns=['Country', 'MSE'])
px.bar(results_df, x='Country', y='MSE', title="Model Error On Each Country", text='MSE')
# for i, country in enumerate(df['Country'].unique()):
#     country_df = df.loc[df.Country.astype(str) == country].reset_index(drop=True)
#     pred = pol.predict(country_df.DayOfYear.to_numpy())
#     fig.add_traces([go.Scatter(x=country_df.DayOfYear, y=country_df.Temp, mode="markers",  marker=dict(color="black", opacity=.7), showlegend=False),
#                     go.Scatter(x=country_df.DayOfYear, y=pred, mode="markers",  marker=dict(color="blue", opacity=.7), showlegend=False)], rows=1, cols=i+1)

# fig.update_layout(title=r"$\text{predicting on different countries}$",
#               margin=dict(t=60),
#               yaxis_title=r"$\widehat{y}$",
#               height=300).show()